In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
code="""
#include <stdio.h>
#include <stdlib.h>

#define N 1024
#define BLOCK_SIZE 16

__global__ void matrixMul(int *a, int *b, int *c, int width) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int sum = 0;
    for (int i = 0; i < width; i++) {
        sum += a[row * width + i] * b[i * width + col];
    }
    c[row * width + col] = sum;
}

int main() {
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = N * N * sizeof(int);

    // Allocate memory on host
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize matrices
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            a[i * N + j] = i + j;
            b[i * N + j] = i - j;
        }
    }

    // Allocate memory on device
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel with 2D grid and 2D block
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 dimGrid((N + dimBlock.x - 1) / dimBlock.x, (N + dimBlock.y - 1) / dimBlock.y);
    matrixMul<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, N);

    // Copy result from device to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Print first and last elements of result
    printf("c[0][0] = %d, c[%d][%d] = %d", c[0], N-1, N-1, c[(N-1) * N + (N-1)]);

    // Free memory
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}
"""

In [4]:
text_file = open('assign4.cu','w')
text_file.write(code)
text_file.close()

In [5]:
!nvcc assign4.cu

In [6]:
!./a.out

c[0][0] = 357389824, c[1023][1023] = -714255872

In [7]:
!nvprof ./a.out

==868== NVPROF is profiling process 868, command: ./a.out
==868== Profiling application: ./a.out
c[0][0] = 357389824, c[1023][1023] = -714255872==868== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   70.71%  9.1149ms         1  9.1149ms  9.1149ms  9.1149ms  matrixMul(int*, int*, int*, int)
                   15.43%  1.9893ms         1  1.9893ms  1.9893ms  1.9893ms  [CUDA memcpy DtoH]
                   13.86%  1.7866ms         2  893.29us  885.88us  900.70us  [CUDA memcpy HtoD]
      API calls:   94.10%  284.67ms         3  94.891ms  107.52us  284.42ms  cudaMalloc
                    4.98%  15.058ms         3  5.0192ms  1.0673ms  12.855ms  cudaMemcpy
                    0.60%  1.8137ms         1  1.8137ms  1.8137ms  1.8137ms  cuDeviceGetPCIBusId
                    0.24%  714.80us         3  238.27us  218.57us  258.02us  cudaFree
                    0.06%  182.81us       101  1.8090us     209ns  75.275us  cuDeviceGe